In [ ]:
# This is very quick and perhaps doesn't need to save anything or at least doesn't need to add days one at a time
# Probably just want to do this once whenever you're evaluating scores
# May make sense ot merge with Evaluate Scores and just separate what you can do before and what you can do after.

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import statsapi
from statsapi import get

import import_ipynb
from Utilities import *

from joblib import Parallel, delayed

import warnings
warnings.simplefilter(action="ignore")

baseball_path = r"C:\Users\james\Documents\MLB\Data"

importing Jupyter notebook from Utilities.ipynb


In [2]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'BBREFTEAM']]

In [3]:
def create_scores(start_date, end_date):
    games = statsapi.schedule(start_date=start_date, end_date=end_date)
    
    all_games = []
    for game in games:
        game_list = [game['game_id'], game['game_type'], game['away_name'], game['home_name'], game['away_score'], game['home_score'], game['game_date']]
        all_games.append(game_list)

    all_games_df = pd.DataFrame(all_games, columns=['game_id', 'game_type', 'away_name', 'home_name', 'away_score', 'home_score', 'game_date'])
    
    all_games_df = all_games_df.query('game_type == "R"')
    
    all_games_df = all_games_df.merge(team_map, left_on='away_name', right_on='FULLNAME', how='left')
    all_games_df = all_games_df.merge(team_map, left_on='home_name', right_on='FULLNAME', how='left')
    
    all_games_df.rename(columns={'BBREFTEAM_x':'away_team', 'BBREFTEAM_y':'home_team','game_date':'date'},inplace=True)
    all_games_df.drop(columns={'FULLNAME_x', 'FULLNAME_y', 'away_name', 'home_name'},inplace=True)
    
    all_games_df['date'] = all_games_df['date'].str.replace("-", "")

    
    return all_games_df

In [4]:
all_games_df = create_scores("04/02/2022", "11/05/2022")
all_games_df

,game_id,game_type,away_score,home_score,date,away_team,home_team
0,663178,R,4,5,20220407,MIL,CHC
1,662766,R,1,3,20220407,CLE,KCR
2,662021,R,0,9,20220407,PIT,STL
3,662571,R,5,1,20220407,NYM,WSN
4,661577,R,6,3,20220407,CIN,ATL
...,...,...,...,...,...,...,...
2474,662789,R,2,3,20221005,PHI,HOU
2475,661427,R,10,1,20221005,MIN,CHW
2476,661422,R,9,12,20221005,ATL,MIA
2477,661227,R,4,2,20221005,ARI,MIL


In [5]:
new = create_scores("03/30/2023", "04/30/2023")
new

,game_id,game_type,away_score,home_score,date,away_team,home_team
0,718780,R,7,2,20230330,ATL,WSN
1,718781,R,0,5,20230330,SFG,NYY
2,718782,R,10,9,20230330,BAL,BOS
3,718777,R,0,4,20230330,MIL,CHC
4,718776,R,0,4,20230330,DET,TBR
...,...,...,...,...,...,...,...
433,718368,R,0,0,20230430,SFG,SDP
434,718357,R,0,0,20230430,CIN,OAK
435,718359,R,0,0,20230430,STL,LAD
436,718354,R,0,0,20230430,PHI,HOU


In [6]:
all_games_df = pd.concat([all_games_df, new], axis=0)
all_games_df.to_csv(os.path.join(baseball_path, 'Scores', 'Scores.csv'))